In [ ]:
import pandas as pd

In [ ]:
train_raw = pd.read_csv("application_train.csv")

In [ ]:
train_raw = pd.DataFrame(train_raw)

In [ ]:
import numpy as np

In [ ]:
(pd.DataFrame(train_raw.describe())).to_csv('train_num_summry.csv')

In [ ]:
train_raw.dtypes[train_raw.dtypes == 'object']

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
print('Training data shape: ', train_raw.shape)
train_raw.head()

In [ ]:
print train_raw['TARGET'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_raw['TARGET'].astype(int).plot.hist()
plt.show()

In [ ]:
# Function to calculate missing values by column# Funct 

def missing_value_table(df):
    mis_value = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_value, mis_val_percent], axis = 1)
    mis_ren_clmn = mis_val_table.rename(columns = {0 : 'Missing Value', 1 : "percent missing"})
    mis_ren_clmn = mis_ren_clmn[mis_ren_clmn.iloc[:,1] !=0].sort_values('percent missing',ascending = False).round(1)
    return mis_ren_clmn


In [ ]:
missing_value_table(train_raw)

In [ ]:
# Number of unique classes in each object column
train_raw.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

In [ ]:
test_raw = pd.DataFrame(pd.read_csv('application_test.csv'))

In [ ]:
print train_raw.shape
print test_raw.shape

In [ ]:
from sklearn import preprocessing

In [ ]:
import sklearn

In [ ]:
# Create a label encoder object
le = preprocessing.LabelEncoder()
le_count = 0

# Iterate through the columns
for col in train_raw:
    if train_raw[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train_raw[col].unique())) <= 2:
            # Train on the training data
            le.fit(train_raw[col])
            # Transform both training and testing data
            train_raw[col] = le.transform(train_raw[col])
            test_raw[col] = le.transform(test_raw[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(train_raw)
app_test = pd.get_dummies(test_raw)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

In [ ]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');
plt.show()

In [ ]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

In [ ]:
# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)



In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment')
plt.show()

In [ ]:
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

In [ ]:
# Find correlations with the target and sort
#correlations = app_train.corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations: \n', correlations.tail(15))
print('\nMost Negative Correlations: \n', correlations.head(15))

In [ ]:
# Find the correlation of the positive days since birth and target
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

In [ ]:
# Set the style of plots
plt.style.use('fivethirtyeight')

# Plot the distribution of ages in years
plt.hist(app_train['DAYS_BIRTH'] / 365, edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count');

In [ ]:
plt.show()

In [ ]:
plt.figure(figsize = (10, 8))

# KDE plot of loans that were repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target == 0')

# KDE plot of loans which were not repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target == 1')

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages')
plt.show()

In [ ]:
# Age information into a separate dataframe
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
age_groups = age_data.groupby('YEARS_BINNED').mean()

In [ ]:
age_groups

In [ ]:
plt.figure(figsize = (8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group')
plt.show()

In [ ]:
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corr = ext_data.corr()
ext_data_corr

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(ext_data_corr, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Copy the data for plotting
plot_data = ext_data.drop(columns = ['DAYS_BIRTH']).copy()

# Add in the age of the client in years
plot_data['YEARS_BIRTH'] = age_data['YEARS_BIRTH']

# Drop na values and limit to first 100000 rows
plot_data = plot_data.dropna().loc[:100000, :]

# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

# Create the pairgrid object
grid = sns.PairGrid(data = plot_data, size = 3, diag_sharey=False,
                    hue = 'TARGET', 
                    vars = [x for x in list(plot_data.columns) if x != 'TARGET'])

# Upper is a scatter plot
grid.map_upper(plt.scatter, alpha = 0.2)

# Diagonal is a histogram
grid.map_diag(sns.kdeplot)

# Bottom is density plot
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

plt.suptitle('Ext Source and Age Features Pairs Plot', size = 32, y = 1.05)
plt.show()

In [ ]:
poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')
poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns = ['TARGET'])

#Imputing missing value
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.fit_transform(poly_features_test)

#Polynomial transformation of degree 5

from sklearn.preprocessing import PolynomialFeatures

poly_transformer = PolynomialFeatures(degree = 5)


In [ ]:
# Train polynomial features

poly_transformer.fit(poly_features)

# Transform the feature
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)

print('Poly train shape : ', poly_features.shape)
print('Poly test shape : ', poly_features_test.shape)

In [ ]:
poly_transformer.get_feature_names \
(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

In [ ]:
# Create a dataframe of the features 
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Add in the target
poly_features['TARGET'] = poly_target

# Find the correlations with the target
poly_corrs = poly_features.corr()['TARGET'].sort_values()



In [ ]:
# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(10))

In [ ]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test)

In [ ]:
poly_transformer.get_feature_names()

In [ ]:
poly_features1 = poly_features.copy()
# drop_columns = list(set(list(app_train.columns)).intersection(list(poly_features.columns)))
# poly_features1.drop(drop_columns, inplace=True, axis=1)
# print poly_features.shape
# print poly_features1.shape

In [ ]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Merge polynomial features into training dataframe
drop_columns = list(set(list(app_train.columns)).intersection(list(poly_features.columns)))
poly_features.drop(drop_columns, inplace=True, axis=1)

poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']

app_train_poly = app_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe

drop_columns_test = list(set(list(app_test.columns)).intersection(list(poly_features_test.columns)))
poly_features_test.drop(drop_columns_test, inplace=True, axis=1)

poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']

app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes
# app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

In [ ]:
list(app_train.columns)

In [ ]:
app_train_domain = app_train_poly.copy()
app_test_domain = app_test_poly.copy()

app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

In [ ]:
# numeric_summary_table_train = pd.DataFrame(app_train_domain[list(app_train_domain.select_dtypes(['int64','float64']).columns)].describe().transpose())
numeric_summary_table_train['colnames'] = app_train_domain.select_dtypes(['int64','float64']).columns
numeric_summary_table_train.columns

In [ ]:
numeric_summary_table_train_maxgt1 = numeric_summary_table_train[numeric_summary_table_train['max'] > 1]

In [ ]:
std_columns_list = list(numeric_summary_table_train_maxgt1['colnames'])
columns_not_considered = ['SK_ID_CURR','CNT_CHILDREN']
std_columns_list = list(set(std_columns_list).difference(set(columns_not_considered)))
std_columns_list

In [ ]:
train = app_train_domain.copy()
test = app_test_domain.copy()
print train.shape
print test.shape

In [ ]:
train_labels = train['TARGET']
# Feature names
train = train.drop('TARGET',axis=1)
features = list(train.columns)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler, Imputer

# Median imputation of missing values
imputer = Imputer(strategy = 'median')


# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(test)

In [ ]:
# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))


# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)


In [ ]:
print train.shape
print test.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

In [ ]:
 voting_clf.fit(train, train_labels)

In [ ]:
train = train.merge(dv, on = 'SK_ID_CURR',how = 'left')